In [2]:
#Расчёт конденсатора кожухотрубного
import matplotlib.pyplot as pt
import numpy as n
from CoolProp.CoolProp import PropsSI as p
from scipy.optimize import root
import math as m

#Исходные данные:
G1 = 636.690003
T11 = 663.629872
T12 = 77

G2 = 543.18
T21 = 60.2275183


P11=3000000
P21=30000000
P22=P21
##############################################
CO2  = 'REFPROP::CO2'
H2O  = 'REFPROP::WATER'
O2   = 'REFPROP::Oxygen'
MIX1 =  'REFPROP::CO2[0.931]&WATER[0.069]'
MIX2 =  'REFPROP::CO2[0.993]&WATER[0.007]'
MIX05 =  'REFPROP::CO2[0.95]&WATER[0.05]' #Для транспортных свойств

H11  = p('H','T', T11+273.15, 'P', P11, MIX1)
H12  = p('H','T', T12+273.15, 'P', P11, MIX1)
H21  = p('H','T', T21+273.15, 'P', P22, MIX2)

#Разбиение по энтальпии:
s    = 100 #Кол-во шагов
step = (H11-H12)/(s) 

#Нулевые массивы:
t1   = n.zeros(s+1)
t2   = n.zeros(s+1)
Q    = n.zeros(s+1)
t1av = n.zeros(s)
t2av = n.zeros(s)
ro1 = n.zeros(s)
ro2 = n.zeros(s)
nu1 = n.zeros(s)
nu2 = n.zeros(s)
lambda1 = n.zeros(s)
lambda2 = n.zeros(s)
Pr1 = n.zeros(s)
Pr2 = n.zeros(s)
DTav = n.zeros(s)
w1 = n.zeros(s)
w2= n.zeros(s)
Re1= n.zeros(s)
Re2= n.zeros(s)
Nu1= n.zeros(s)
Nu2= n.zeros(s)
f1= n.zeros(s)
f2= n.zeros(s)
fNu1= n.zeros(s)
fNu2= n.zeros(s)
alpha1= n.zeros(s)
alpha2= n.zeros(s)
K= n.zeros(s)
F= n.zeros(s+1)
L= n.zeros(s+1)
P2= n.zeros(s+1)
P1= n.zeros(s+1)
tw2 = n.zeros(s+1)
t1av = n.zeros(s)
t2av = n.zeros(s)
ro1 = n.zeros(s)
ro2 = n.zeros(s)
nu1 = n.zeros(s)
nu2 = n.zeros(s)
lambda1 = n.zeros(s)
lambda2 = n.zeros(s)
Pr1 = n.zeros(s)
Pr2 = n.zeros(s)
DTav = n.zeros(s)
w1 = n.zeros(s)
w2= n.zeros(s)
Re1= n.zeros(s)
Re2= n.zeros(s)
Nu1= n.zeros(s)
Nu2= n.zeros(s)
f1= n.zeros(s)
f2= n.zeros(s)
alpha1= n.zeros(s)
alpha2= n.zeros(s)
K= n.zeros(s)
F= n.zeros(s+1)
L= n.zeros(s+1)
P1= n.zeros(s+1)
P2= n.zeros(s+1)
Ntt= n.zeros(s+1)

ddP1     =n.zeros(10000)
ddP2     =n.zeros(10000)
ddP4     =n.zeros(10000)
ddP6     =n.zeros(10000)
ddP8     =n.zeros(10000)
ddP10    =n.zeros(10000)




Vv     =n.zeros(10000000)
DdP1   =n.zeros(10000000)
DdP2   =n.zeros(10000000)
Ff     =n.zeros(10000000)
Ll     =n.zeros(10000000)



#Противоток:
h11 = H11
i=0
for i in range(s+1):
    t1[i] = p('T','H', h11, 'P', P11, MIX1)-273.15 
    h12 = h11 - step
    dQ = G1 * (h11-h12)
    if i < s:
        Q[i+1]=Q[i]+dQ
    h11 = h12
h21 = H21
i=0
for i in range(s+1):
    t2[s-i] = p('T','H', h21, 'P', P22, MIX2)-273.15  
    h22 = h21+((Q[s-i]-Q[s-i-1])/G2)
    h21 = h22
DT=t1-t2



#Геометрия:
Dvnes = 0.016 #Внешний диаметр трубки https://docs.cntd.ru/document/1200001411
DeltaTube = 0.001
DeltaKorp = 0.02
Dvnut = Dvnes-2*DeltaTube
steptube = Dvnes*1.2
lambdaw = 20
delta = 10**-5 #Шероховатость
dh1 = Dvnes
dh2 = Dvnut


j=0
wout2go = 0.01
wout2stop = 100
wout2step = 0.02
for wout2 in n.arange(wout2go,wout2stop,wout2step):
    
    #Количество труб по диаметру:
    roout2 = p('D','T', t2[s]+273.15, 'P', P22, MIX2)
    Fproh2 = G2/(wout2*roout2)
    
    Ntubes = m.ceil(Fproh2/((n.pi*Dvnut**2)/4))
    
    summ = 0
    nd=0
    n2 = 0
    while n2 < Ntubes:
        summ = summ + nd
        n2=1+6*summ
        nd=nd+1
    nd = 1+2*(nd-1)
    Ntubes = n2
    Dkorp = nd*steptube+2*Dvnes
    Fproh1 = ((n.pi*Dkorp**2)/4)-(Ntubes*(n.pi*Dvnes**2)/4)
    Fl = n.pi*((Dvnes+Dvnut)/2)*Ntubes
    
    P2[0]=P22
    P1[0]=P11
    
    #Зона охлаждения:
    for i in range(s):
        #Свойства
        t1av[i]   =(t1[i]+t1[i+1])/2
        t2av[i]   =(t2[i]+t2[i+1])/2
        ro1[i]    =p('D','T', t1av[i]+273.15, 'P', P11, MIX1)
        ro2[i]    =p('D','T', t2av[i]+273.15, 'P', P21, MIX2)
        nu1[i]    =p('V','T', t1av[i]+273.15, 'P', P11, MIX05)/ro1[i]
        nu2[i]    =p('V','T', t2av[i]+273.15, 'P', P21, MIX05)/ro2[i]
        lambda1[i]=p('L','T', t1av[i]+273.15, 'P', P11, MIX05)
        lambda2[i]=p('L','T', t2av[i]+273.15, 'P', P21, MIX05)
        Pr1[i]    =p('PRANDTL','T', t1av[i]+273.15, 'P', P11, MIX05)
        Pr2[i]    =p('PRANDTL','T', t2av[i]+273.15, 'P', P21, MIX05)
        if abs(Pr1[i]-Pr1[i-1])>Pr1[i-1]:
            Pr1[i]=p('PRANDTL','T', t1[i]+273.15, 'P', P11, CO2)
        if abs(Pr2[i]-Pr2[i-1])>Pr2[i-1]:
            Pr2[i]=p('PRANDTL','T', t2[i]+273.15, 'P', P21, CO2)
        DTav[i]   =t1av[i]-t2av[i]
        w1[i] = G1/(ro1[i]*Fproh1)
        w2[i] = G2/(ro2[i]*Fproh2)
        Re1[i] = w1[i]*dh1/nu1[i]
        Re2[i] = w2[i]*dh2/nu2[i]
        Nu2[i] = 0.021*Re2[i]**0.8*Pr2[i]**0.43
        def func(x):
            delta = 10**-5
            left = x
            right = 1/(2*n.log10((2.51/(Re2[i]*(x)**(1/2)))+(delta/3.7)))**2
            return left-right
        sol = root(func, 0.01)
        f2[i]=float(sol.x)
        def func(x):
            delta = 10**-5
            left = x
            right = 1/(2*n.log10((2.51/(Re1[i]*(x)**(1/2)))+(delta/3.7)))**2
            return left-right
        sol = root(func, 0.01)
        f1[i]=float(sol.x)
        alpha2[i] = Nu2[i]*lambda2[i]/dh2
        w1[i] = G1/(ro1[i]*Fproh1)
        Re1[i] = w1[i]*Dvnes/nu1[i]
        Nu1[i] = 0.22*(Re1[i]**0.65)*(Pr1[i]**0.36)
        alpha1[i] = Nu1[i]*lambda1[i]/dh1
        K[i] = 1/((1/alpha1[i])+(1/alpha2[i])+((((Dvnes+Dvnut)/2)/(2*lambdaw))*(n.log(Dvnes/Dvnut))))
        dF = (Q[i+1]-Q[i])/(K[i]*DTav[i])
        dL = dF/Fl
        if i < s:
            F[i+1]=F[i]+dF
        if i < s:
            L[i+1]=L[i]+dL
        dP2 = f2[i]*dL/dh2*ro2[i]*w2[i]**2/2
        dP1 = f1[i]*dL/dh1*ro1[i]*w1[i]**2/2
        if i < s:
            P1[i+1]=P1[i]-dP1
            P2[i+1]=P2[i]-dP2
    DP1 = P1[0]-P1[s]
    DP2 = P2[0]-P2[s]
    Vv[j] = L[s]*n.pi*(Dkorp**2)/4
    
    if round(DP1/1000000000000)==round(0.01*P11/1000000000000):     #0.01 - процент от входного 
        ddP1[j] = DP1
    if round(DP1/1000000000000)==round(0.02*P11/1000000000000):     #0.02 - процент от входного 
        ddP2[j] = DP1
    if round(DP1/1000000000000)==round(0.04*P11/1000000000000):     #0.04 - процент от входного 
        ddP4[j] = DP1
    if round(DP1/1000000000000)==round(0.06*P11/1000000000000):     #0.06 - процент от входного 
        ddP6[j] = DP1
    if round(DP1/1000000000000)==round(0.08*P11/1000000000000):     #0.08 - процент от входного 
        ddP8[j] = DP1
    if round(DP1/1000000000000)==round(0.1*P11/1000000000000):     #0.1 - процент от входного 
        ddP10[j] = DP1    
    
    DdP1[j]   = DP1
    DdP2[j]   = DP2
    Ff[j]     = F[s]
    Ll[j]     = L[s]
    j=j+1
    Ntt[j] = Ntubes
    if DP1>0.15*P11:
        break;
imax=j
num = int((wout2-wout2go)/wout2step)


dP1p = [10**5 for i in range(imax)]
dP2p = [10**5 for i in range(imax)]
dP4p = [10**5 for i in range(imax)]
dP6p = [10**5 for i in range(imax)]
dP8p = [10**5 for i in range(imax)]
dP10p = [10**5 for i in range(imax)]

for i in range(imax):
    dP1p[i] = abs(0.01*P11 - ddP1[i])
j = n.argmin(dP1p)
print('1%', DdP1[j],DdP2[j],Ff[j], Vv[j])

for i in range(imax):
    dP2p[i] = abs(0.02*P11 - ddP2[i])
j = n.argmin(dP2p)
print('2%', DdP1[j],DdP2[j],Ff[j], Vv[j])

for i in range(imax):
    dP4p[i] = abs(0.04*P11 - ddP4[i])
j = n.argmin(dP4p)
print('4%', DdP1[j],DdP2[j],Ff[j], Vv[j],Ll[j],Ntt[j])

for i in range(imax):
    dP6p[i] = abs(0.06*P11 - ddP6[i])
j = n.argmin(dP6p)
print('6%', DdP1[j],DdP2[j],Ff[j], Vv[j])

for i in range(imax):
    dP8p[i] = abs(0.08*P11 - ddP8[i])
j = n.argmin(dP8p)
print('8%', DdP1[j],DdP2[j],Ff[j], Vv[j])

for i in range(imax):
    dP10p[i] = abs(0.1*P11 - ddP10[i])
j = n.argmin(dP10p)
print('10%', DdP1[j],DdP2[j],Ff[j], Vv[j])
print(' ')

1% 31608.973384339828 3919.268166065216 54892.66471254369 458.8054317518383
2% 59794.327985527925 7597.548729110509 43743.24929715587 366.86012117909394
4% 119389.5702203121 15558.128997523338 34357.14081117318 289.3859518758687 68.64524734773879 10981.0
6% 184287.54082555557 24491.897991620004 29563.158506555577 249.7774725395229
8% 234074.57812878955 31050.71366360411 27340.32468558371 231.41717966597574
10% 300144.3278821013 40716.63397434726 25044.386467641227 212.41763840792973
 
